# Assignment 2: Implementing Decoding Strategies for Summarization (40 points)

In this assignment, you will **implement decoding algorithms** used for text summarization using a pretrained Transformer model.

---

### Your Task
1. **Load** the `sshleifer/distilbart-cnn-12-6` summarization model.
2. **Implement the following decoding strategies from scratch** (no `.generate()` allowed!). You need to provide your own explanation on your implementation for each function:
   - Greedy decoding (3 points)
   - Top-k sampling (3 points)
   - Top-p (nucleus) sampling (3 points)
   - Beam search (3 points)
   - Beam search with n-gram blocking (3 points)
3. Use your decoder to summarize 200 articles from the CNN/DailyMail dataset.
4. Implement the following ROUGE metrics and evaluate your summaries using your own ROUGE metric implementation.
  - Implementation
    - ROUGE-n (2 points): e.g., ROUGE-1 & ROUGE-2
    - ROUGE-L (4 points)
  - Explanation on your implementation (2 points)
  - Discuss how to improve these metrics to perform a better evaluation? (2 points)
5. Discussion (15 points)

---

**Note:**
  - Regarding the decoding strategies, you are expected to work directly with model logits and sampling logic. Do not use `model.generate()` or any pre-built function. **Hint**: use "outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)"
  - For each function, you need to write clear and concise comments about your implementation. This may not be line-by-line, but rather meaningful chunk of codes.
  - For each question, justify your answer with explanations.

### **We highly recommend using 'cpu' as the default for development, and switching to 'gpu' only for evaluating summarization performance. This is due to the limited GPU availability in the Colab environment.**


In [1]:
!pip install transformers datasets --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cu

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import torch
import json
import torch.nn.functional as F
import random
import re
from collections import Counter

# Load model and tokenizer
model_name = "sshleifer/distilbart-cnn-12-6"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# load model to gpu for faster inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")
model.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Using device: cuda


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In this assignment, we use the CNN/DailyMail summarization dataset, a widely-used benchmark for training and evaluating text summarization models.

Each data sample consists of:

- article: A news story, usually between 300 and 800 words.

- highlights: A bullet-style abstractive summary of the article, written by human editors.

This dataset is ideal for testing decoding strategies because:

- The summaries are relatively short and factual

- The dataset is large enough to support diverse decoding behavior

- It has been used in many summarization papers, making ROUGE scores easy to compare

In this assignment, we will use a small subset of 200 samples from the validation set for quick experimentation and evaluation.

In [3]:
# Upload cnn_dm_200.json file

# Load 200-sample dataset
with open("cnn_dm_200.json", "r") as f:
    dataset = json.load(f)

print(f"Loaded {len(dataset)} samples.")
print("\nExample article:")
print(dataset[0]["article"])
print("\nReference summary:")
print(dataset[0]["highlights"])


Loaded 200 samples.

Example article:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki

## Decoding in Language Models
Pretrained language models like BART or GPT generate text by predicting the most likely next token one at a time.
However, simply choosing the most probable token at each step often leads to deterministic, repetitive, or generic outputs.

To overcome this, several decoding strategies have been proposed to balance:

- fluency vs. diversity

- coherence vs. novelty

In this assignment, you will implement five widely-used decoding strategies from scratch and compare their effects.

### Greedy Decoding

In [5]:
def greedy_decode(input_ids, max_length=128):
    """
    Perform greedy decoding from the model using logits.

    Args:
        input_ids (torch.Tensor): Tokenized input tensor of shape [1, seq_len].
        max_length (int): Maximum length of the generated sequence.

    Returns:
        str: Decoded summary text.
    """
    # TODO: Implement Greedy Decoding from scratch
    input_ids = input_ids.to(device) # load model to device
    decoder_input_ids = torch.tensor([[model.config.decoder_start_token_id]]).to(device) # take the start token for decoding
    eos_token_id = model.config.eos_token_id # take the EOS token id from the vocab

    for _ in range(max_length):

        with torch.no_grad():
            output = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)
        proba_output = torch.softmax(output.logits[:,-1,:],axis=-1) # take the last token logits and apply softmax to get the probability of each token
        id = torch.tensor([[torch.argmax(proba_output, axis=-1)]]).to(device) # greedy approach to take the token_id with largest probability
        decoder_input_ids = torch.cat([decoder_input_ids,id],axis=-1) # append the decoded token to the generated token sequence

        if id.item() == eos_token_id: # If the model generate an EOS token, the decoding process should be stop
            break

    summary = tokenizer.decode(decoder_input_ids[0], skip_special_tokens=True) # convert the generated token sequence to string

    return summary


Explain your implementation

My implementation performs greedy decoding for a sequence-to-sequence model by iteratively generating tokens based on the highest probability prediction at each step. Firstly, it inputs a decoder start token id to start auto-regressive process. The model takes in decoder start token id and returns logits for next token. The softmax function is applied to convert logits to probabilites, and the token id, which is the index in the tensor, is chosen with respect to the highest probability. The chosen token id is then concatenated with the sequence of previously generated tokens, to feed back into the model for the next iteration. The process stop until model outputs the end of sentence token.


### Top-k Decoding

In [6]:
def top_k_decode(input_ids, k=2, max_length=128, temperature=1.0):
    """
    Perform Top-k sampling decoding from the model.

    Args:
        input_ids (torch.Tensor): Tokenized input tensor of shape [1, seq_len].
        k (int): Number of top tokens to sample from.
        max_length (int): Maximum length of the generated sequence.
        temperature (float): Softmax temperature for sampling.

    Returns:
        str: Decoded summary text.
    """
    # TODO: Implement Top-k Sampling Decoding
    input_ids = input_ids.to(device)
    decoder_input_ids = torch.tensor([[model.config.decoder_start_token_id]]).to(device) # take the start token for decoding
    eos_token_id = model.config.eos_token_id # take the EOS token id from the vocab

    for _ in range(max_length):
      with torch.no_grad():
        output = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)
      output_proba = torch.softmax(output.logits[:,-1,:]/temperature,axis=-1) # convert logits to probability, devide by the temperature to control the randomness and shape of probability distribution
      values, indices = torch.topk(output_proba, k=k, axis=-1) # take top k tokens with largest probability
      values = values/ torch.sum(values,axis=-1,keepdim=True).item() # redistribute probability of selected tokens, made them summing up to 1
      local_index = torch.multinomial(values[0],num_samples=1) # sampling from redistributed probability distribution
      id = indices[:,local_index] # take the token id of the sampled token
      decoder_input_ids = torch.cat([decoder_input_ids,id],axis=-1) # add the sampled token to the generated token sequence

      if id.item() == eos_token_id: # If the model generate an EOS token, the decoding process should be stop
        break

    summary = tokenizer.decode(decoder_input_ids[0], skip_special_tokens=True) # convert the generated token sequence to string
    return summary




- Explain your implementation.
- What are the role of hyperparameters of this strategy? and what happen if you change the values?

- My implementation of top-k sampling involves selecting the top k most probable tokens from the model's output logits at each decoding step. The logits are first converted to probabilities using the softmax function. Then, use torch.topk function to select k indices with largest probabilities. Then, the probabilities of k selected tokens must be redistributed to have sum of 1. After that, I sample a token using torch.multinomial function to sample elements from a given probability distribution. This process is repeated until an end-of-sequence token is generated or a maximum length is reached.
- The hyperparameter k determines the number of top tokens to consider for sampling. A smaller k leads to more deterministic outputs, as fewer options are available, while a larger k increases diversity but may introduce noise. If k is set too high, the model may generate less coherent or relevant text, as it has too many options to choose from. Conversely, if k is too low, the generated text may become repetitive or lack variety. If k is set to 1, top k sampling becomes equivalent to greedy decoding, as only the most probable token is selected at each step.
- The hyperpatameter temperature is used to control the randomness of the sampling process. A lower temperature results in more conservative and deterministic outputs, while a higher temperature increases diversity but may lead to less coherent text. If the temperature is set to 1, it means no scaling is applied to the logits, and the sampling process behaves as if it were using the original probabilities. If we increase the temperature, the probabilities distribution becomes more uniform. In contrast, we reduce the temperature, the distribution becomes sharper.

### Top-p Decoding

In [11]:
def top_p_decode(input_ids, p=0.9, max_length=128, temperature=1.0):
    """
    Perform Top-p (nucleus) sampling decoding from the model.

    Args:
        input_ids (torch.Tensor): Tokenized input tensor of shape [1, seq_len].
        p (float): Cumulative probability threshold for sampling.
        max_length (int): Maximum length of the generated sequence.
        temperature (float): Softmax temperature for sampling.

    Returns:
        str: Decoded summary text.
    """
    # TODO: Implement Top-p Sampling Decoding

    input_ids = input_ids.to(device)
    decoder_input_ids = torch.tensor([[model.config.decoder_start_token_id]]).to(device) # take the start token for decoding
    eos_token_id = model.config.eos_token_id # take the EOS token id from the vocab

    for _ in range(max_length):
      with torch.no_grad():
        output = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)
      output_proba = torch.softmax(output.logits[:,-1,:]/temperature,axis=-1)[0] # convert logits to probability, devide by the temperature to control the randomness and shape of probability distribution
      sorted_probs, sorted_indices = torch.sort(output_proba, descending=True) # sort the probabilites and their corresponding token ids
      cumulative_probs = torch.cumsum(sorted_probs, dim=0) # calculate the cumulative probability
      nucleus_mask = cumulative_probs <= p # create a mask for the tokens that are below the threshold
      nucleus_mask[max((cumulative_probs > p).nonzero(as_tuple=True)[0][0], 0)] = True # Ensure we include at least one token after passing threshold
      nucleus_probs = sorted_probs[nucleus_mask] # filter the probabilities based on the mask
      nucleus_indices = sorted_indices[nucleus_mask] # filter the token ids based on the mask
      nucleus_probs = nucleus_probs/torch.sum(nucleus_probs) # redistribute the probabilities of the selected tokens
      local_index = torch.multinomial(nucleus_probs,num_samples=1) # sampling from redistributed probability distribution
      id = torch.tensor([[nucleus_indices[local_index]]]).to(device)
      decoder_input_ids = torch.cat([decoder_input_ids,id],axis=-1)

      if id.item() == eos_token_id: # If the model generate an EOS token, the decoding process should be stop
        break


    summary = tokenizer.decode(decoder_input_ids[0], skip_special_tokens=True)
    return summary



- Explain your implementation.
- What are the role of hyperparameters of this strategy? and what happen if you change the values?

- My implementation of top-p (percentile) decoding is different from top-k sampling in that: It selects the smallest set of tokens whose cumulative probability exceeds a threshold p. The logits are first devided by temperature to control the shape of distribution and converted to probabilities using the softmax function. Then, I sort the probabilities in descending order and compute the cumulative sum. I select the smallest set of tokens such that their cumulative probability is greater than or equal to p, implemented by using masks for filtering in tensor. After that, the probabilities of selected candidates must be redistributed to sum up 1. Afterwards, I sample a token from this set and the remaining process is similar to the top-k sampling method. This process is repeated until an end-of-sequence token is generated or a maximum length is reached.      
- The hyperparameter p determines the threshold for cumulative probability. A smaller p leads to more deterministic outputs, as fewer options are available, while a larger p increases diversity but may introduce noise. If p is set too high, the model may generate less coherent or relevant text, as it has too many options to choose from. Conversely, if p is too low, the generated text may become repetitive or lack variety.
- The hyperpatameter temperature is used to control the randomness of the sampling process. A lower temperature results in more conservative and deterministic outputs, while a higher temperature increases diversity but may lead to less coherent text. If the temperature is set to 1, it means no scaling is applied to the logits, and the sampling process behaves as if it were using the original probabilities. If we increase the temperature, the probabilities distribution becomes more uniform. In contrast, we reduce the temperature, the distribution becomes sharper.


### Beam Search Decoding

In [8]:
def beam_search_decode(input_ids, beam_size=4, max_length=128):
    """
    Perform beam search decoding from the model.

    Args:
        input_ids (torch.Tensor): Tokenized input tensor of shape [1, seq_len].
        beam_size (int): Number of beams to explore.
        max_length (int): Maximum length of the generated sequence.

    Returns:
        str: Decoded summary text.
    """
    # TODO: Implement Beam Search Decoding
    input_ids = input_ids.to(device)
    decoder_input_ids = torch.tensor([[model.config.decoder_start_token_id]]).to(device) # take the start token for decoding
    eos_token_id = model.config.eos_token_id # take the EOS token id from the vocab
    vocab_size = model.config.vocab_size # take the vocab size from the model config

    top_beam = [] # to store top k beams with highest probability at each step
    with torch.no_grad():
      output = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)

    output_proba = torch.softmax(output.logits[:,-1,:],axis=-1)
    probas, indices = torch.topk(output_proba,k=beam_size,axis=-1) # select top k tokens with largest probability, called beams in the first step
    for index,proba in zip(indices[0],probas[0]):
      top_beam.append((torch.tensor([[decoder_input_ids,index]]),proba.item())) # append top k beams to the list to start the beam search process

    for _ in range(max_length):
      beam_list = [] # beam list to store all the beams, total number of beams in comparison will be beam_size * vocab_size
      for beam in top_beam:
        with torch.no_grad():
          output = model(input_ids=input_ids, decoder_input_ids=beam[0]).logits[:,-1,:]
        output_proba = beam[1] * torch.softmax(output,axis=-1) # calculate the cumulative probability by multiplying the probability of the current beam with the conditional probability of the next token
        for idx,local_proba in enumerate(list(output_proba[0])):
          beam_list.append((torch.cat([beam[0],torch.tensor([[idx]])],axis=-1), local_proba.item())) # append each candidate to the beam list, the candidate is the current beam with the next token id appended to it

      sorted_beam_list = sorted(beam_list, key=lambda item: item[1], reverse=True) # sort the beam list based on the cumulated probability for later selection
      top_beam = sorted_beam_list[:beam_size] # select top k beams with largest probability

      if top_beam[0][0][0][-1].item() == eos_token_id:
        break

    summary = tokenizer.decode(top_beam[0][0][0], skip_special_tokens=True)
    return summary



- Explain your implementation.
- What are the role of hyperparameters of this strategy? and what happen if you change the values?

- My implementation of beam search decoding involves maintaining a fixed number of candidate sequences (beams) at each decoding step. The process starts with the decoder start token id and iteratively expands each beam by generating the next token probabilities. For each beam, I compute the probabilities of all possible next tokens and select the top k beams based on their cumulative probabilities. This is done using append all possible beams to the list and sorted based on their probabilities. The process continues until an end-of-sequence token is generated and is selected as the highest probable beam or a maximum length is reached.
- The hyperparameter k (beam width) determines the number of candidate sequences to maintain at each step. A larger k increases the search space and may result in more diverse output as it can explore beam with higher probability. This may leads to more coherent outputs, but also increases computational cost. A smaller k reduces the search space and may lead to less diverse outputs. If k is set to 1, beam search becomes equivalent to greedy decoding, as only the most probable sequence is selected at each step.

### Beam Search with N-gram Blocking

In [10]:

def isBlocked(token_seq: list(), no_repeat_ngram_size: int, next_token: int):
  '''
  This function checks if the next token is blocked by the n-gram repetition blocking rule.
  It iteratively check if the next token is in the last n-gram size of the token sequence.
  '''
  if len(token_seq) < no_repeat_ngram_size:
    return False
  candidate_seq = token_seq[-(no_repeat_ngram_size-1):] + [next_token]

  for i in range(len(token_seq)-(no_repeat_ngram_size-1)):
    ref_seq = token_seq[i: i+ no_repeat_ngram_size]
    if ref_seq == candidate_seq:
      return True
  return False

def beam_search_ngram_block(input_ids, beam_size=4, max_length=128, no_repeat_ngram_size=3):
    """
    Perform beam search decoding with n-gram repetition blocking.

    Args:
        input_ids (torch.Tensor): Tokenized input tensor of shape [1, seq_len].
        beam_size (int): Number of beams to explore.
        max_length (int): Maximum length of the generated sequence.
        no_repeat_ngram_size (int): Size of n-gram to prevent from repeating.

    Returns:
        str: Decoded summary text.
    """
    # TODO: Implement Beam Search with n-gram blocking


    input_ids = input_ids.to(device)
    decoder_input_ids = torch.tensor([[model.config.decoder_start_token_id]]).to(device)
    eos_token_id = model.config.eos_token_id # take the EOS token id from the vocab
    vocab_size = model.config.vocab_size # take the vocab size from the model config

    top_beam = []
    with torch.no_grad():
      output = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids).logits[:,-1,:]
    output_proba = torch.softmax(output,axis=-1)
    probas, indices = torch.topk(output_proba,k=beam_size,axis=-1)
    for index,proba in zip(indices[0],probas[0]):
      top_beam.append((torch.tensor([[decoder_input_ids,index]]),proba.item()))

    for _ in range(max_length):
      beam_list = []
      for beam in top_beam:
        with torch.no_grad():
          output = model(input_ids=input_ids, decoder_input_ids=beam[0].to(device))
        output = output.logits[:,-1,:]
        output_proba = beam[1] * torch.softmax(output,axis=-1)
        for idx,local_proba in enumerate(list(output_proba[0])):
          if not isBlocked(token_seq=beam[0][0].tolist(),no_repeat_ngram_size=no_repeat_ngram_size,next_token=idx): # check if the next token is blocked by the n-gram repetition blocking rule before being added to the beam list
            beam_list.append((torch.cat([beam[0],torch.tensor([[idx]])],axis=-1), local_proba.item()))
      sorted_beam_list = sorted(beam_list, key=lambda item: item[1], reverse=True)
      top_beam = sorted_beam_list[:beam_size]

      if top_beam[0][0][0][-1].item() == eos_token_id:
        break

    summary = tokenizer.decode(top_beam[0][0][0], skip_special_tokens=True)
    return summary

- Explain your implementation.
- What are the role of hyperparameters of this strategy? and what happen if you change the values?

- My implementation for beam search with n-gram blocking is almost similar to the implementation of beam search decoding, except the checking of valid beams. before the candidate beam is added to the list, it should be checked if it creates n-grams that are already present in the previous beams. If it does, it is discarded.
- The hyperparameter n (no_repeat_ngram_size) determines the size of the n-grams to block. A larger n leads to less aggressive blocking as only large n-grams are blocked, allowing short repetition. A smaller n avoids more repetition.

In [ ]:
# Example usage with the above decoding functions
input_text = dataset[0]['article']
input_ids = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=1024).input_ids

print("Greedy Search:")
print(greedy_decode(input_ids))

print("\nTop k decoding:")
print(top_k_decode(input_ids))

print("\nTop p decoding:")
print(top_p_decode(input_ids))

print("\nBeam search decoding:")
print(beam_search_decode(input_ids))

print("\nBeam Search + N-gram Blocking:")
print(beam_search_ngram_block(input_ids))

## ROUGE Metric Evaluation

In [ ]:


def tokenize(text):
    return re.findall(r'\w+', text.lower())

def compute_rouge_n(reference: str, generated: str, n: int = 1) -> dict:
    """
    Compute ROUGE-N score between reference and generated text.

    Args:
        reference (str): The reference summary.
        generated (str): The generated summary.
        n (int): The n-gram size (e.g., 1 for ROUGE-1).

    Returns:
        dict: Dictionary with 'precision', 'recall', and 'f1' scores.
    """
    ref_tokens = tokenize(reference)
    gen_tokens = tokenize(generated)

    ref_ngrams = Counter([tuple(ref_tokens[i:i+n]) for i in range(len(ref_tokens)-n+1)])
    gen_ngrams = Counter([tuple(gen_tokens[i:i+n]) for i in range(len(gen_tokens)-n+1)])


    overlap = sum((ref_ngrams & gen_ngrams).values())

    precision = overlap / sum(gen_ngrams.values()) if gen_ngrams else 0.0
    recall = overlap / sum(ref_ngrams.values()) if ref_ngrams else 0.0
    f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0.0

    return {'precision': precision, 'recall': recall, 'f1': f1}

def lcs(X, Y):
    """
    Helper function to compute the length of Longest Common Subsequence (LCS)
    """
    m, n = len(X), len(Y)
    dp = [[0] * (n+1) for _ in range(m+1)]

    for i in range(m):
        for j in range(n):
            if X[i] == Y[j]:
                dp[i+1][j+1] = dp[i][j] + 1
            else:
                dp[i+1][j+1] = max(dp[i][j+1], dp[i+1][j])
    return dp[m][n]

def compute_rouge_l(reference: str, generated: str) -> dict:
    """
    Compute ROUGE-L (Longest Common Subsequence) score.

    Args:
        reference (str): The ground truth summary.
        generated (str): The generated summary by the model.

    Returns:
        dict: Dictionary with precision, recall, and f1 scores.
    """
    ref_tokens = tokenize(reference)
    gen_tokens = tokenize(generated)

    lcs_len = lcs(ref_tokens, gen_tokens)

    precision = lcs_len / len(gen_tokens) if gen_tokens else 0.0
    recall = lcs_len / len(ref_tokens) if ref_tokens else 0.0
    f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0.0

    return {'precision': precision, 'recall': recall, 'f1': f1}


- Explanation on your implementation
- Discuss how to improve these metrics to perform a better evaluation?

## Evaluation

In [ ]:
def generate_summary_custom(article_text, strategy="greedy"):
    input_ids = tokenizer(article_text, return_tensors="pt", truncation=True, max_length=1024).input_ids
    if strategy == "greedy":
        return greedy_decode(input_ids)
    elif strategy == "top_k":
        return top_k_decode(input_ids)
    elif strategy == "top_p":
        return top_p_decode(input_ids)
    elif strategy == "beam":
        return beam_search_decode(input_ids)
    elif strategy == "beam_block":
        return beam_search_ngram_block(input_ids)
    else:
        raise ValueError("Unknown decoding strategy")


In [ ]:
# Compare summaries across all decoding strategies for one article
sample_article = dataset[0]["article"]
for strategy in ["greedy", "top_k", "top_p", "beam", "beam_block"]:
    print(f"\n[{strategy.upper()}]")
    print(generate_summary_custom(sample_article, strategy=strategy))



In [ ]:
import pandas as pd
import numpy as np

# Decoding strategies
strategies = ["greedy", "top_k", "top_p", "beam", "beam_block"]

# Save evalution results
results = {s: [] for s in strategies}

# Evaluation loop
for i, sample in enumerate(dataset):
    article = sample["article"]
    reference = sample["highlights"]

    input_ids = tokenizer(article, return_tensors="pt", truncation=True, max_length=1024).input_ids

    for strategy in strategies:
        try:
            generated = generate_summary_custom(article, strategy=strategy)

            rouge1 = compute_rouge_n(reference, generated, n=1)["f1"]
            rouge2 = compute_rouge_n(reference, generated, n=2)["f1"]
            rougel = compute_rouge_l(reference, generated)["f1"]

            results[strategy].append({
                "rouge1": rouge1,
                "rouge2": rouge2,
                "rougeL": rougel
            })
        except Exception as e:
            print(f"[{strategy}] Error on sample {i}: {e}")


In [ ]:
summary = {}
for strategy in strategies:
    if results[strategy]:
        rouge1s = [x["rouge1"] for x in results[strategy]]
        rouge2s = [x["rouge2"] for x in results[strategy]]
        rougels = [x["rougeL"] for x in results[strategy]]

        summary[strategy] = {
            "ROUGE-1": np.mean(rouge1s),
            "ROUGE-2": np.mean(rouge2s),
            "ROUGE-L": np.mean(rougels)
        }

df = pd.DataFrame(summary).T.sort_values("ROUGE-L", ascending=False)
display(df)


## Discussion (Total: 15 points)
Answer the following questions based on your decoding outputs and analysis. Be clear and support your claims with examples. You may provide your evidence by implementing additional functions. For example, you may draw a plot or show a statistics.

1. Compare the different decoding strategies. Present and justify your findings using examples from your generated summaries. (9 points)
  - Which strategies produce more diverse outputs?

  - Which ones tend to repeat phrases or truncate early?

  - Which are more stable across different runs?


2. Analyze the impact of decoding parameters. Suggest complementary evaluation methods that might improve reliability. (3 points)
  - How does increasing beam size or sampling temperature affect the results?

  - Use at least one example to illustrate the effect.

3. Discuss the limitations of ROUGE as an evaluation metric. Suggest complementary evaluation methods that might improve reliability. (3 points)
  - What aspects of summarization quality does ROUGE fail to capture?


